In [1]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
import keras.layers
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import os
from os import listdir
import PIL
from PIL import Image
from PIL import ImageOps
from imutils import paths

import numpy as np

In [2]:
image_size = (224, 224)
dropout = 0.2
batch_size = 32
lr = 1e-4
epochs = 20

In [3]:
maskoff_paths = list(paths.list_images('faces/maskoff/'))
maskon_paths = list(paths.list_images('faces/maskon/'))

data = []
labels = []

# maskoff
for maskoff_path in maskoff_paths:
    label = 'maskoff'
    image = load_img(maskoff_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)
    labels.append(label)

# maskon
for maskon_path in maskon_paths:
    label = 'maskon'
    image = load_img(maskon_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)
    labels.append(label)

In [4]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [5]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = keras.utils.to_categorical(labels)

In [6]:
(train_x, test_x, train_y, test_y) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

In [7]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=image_size + (3,))
head_model = base_model.output
head_model = keras.layers.AveragePooling2D(pool_size=(7, 7))(head_model)
head_model = keras.layers.Flatten(name="flatten")(head_model)
head_model = keras.layers.Dense(128, activation="relu")(head_model)
head_model = keras.layers.Dropout(0.5)(head_model)
head_model = keras.layers.Dense(2, activation="softmax")(head_model)

model = keras.models.Model(inputs=base_model.input, outputs=head_model)
for layer in base_model.layers:
	layer.trainable = False

In [8]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")
train_generator = aug.flow(
    train_x, train_y,
    batch_size = batch_size
)

In [9]:
callbacks = [
    keras.callbacks.ModelCheckpoint("saved_models/RGB_3/save_at_{epoch}.keras")
]

In [10]:
opt = keras.optimizers.Adam(learning_rate=lr, decay = lr / epochs)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [11]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_x) // batch_size,
	validation_data=(test_x, test_y),
	validation_steps=len(test_x) // batch_size,
	epochs=epochs,
    callbacks=callbacks)

Epoch 1/20
47/47 [==============================] - 37s 697ms/step - loss: 0.7574 - accuracy: 0.5989 - val_loss: 0.5559 - val_accuracy: 0.7910
Epoch 2/20
47/47 [==============================] - 29s 621ms/step - loss: 0.5785 - accuracy: 0.7283 - val_loss: 0.4493 - val_accuracy: 0.8598
Epoch 3/20
47/47 [==============================] - 30s 629ms/step - loss: 0.5014 - accuracy: 0.7886 - val_loss: 0.3831 - val_accuracy: 0.8995
Epoch 4/20
47/47 [==============================] - 30s 632ms/step - loss: 0.4266 - accuracy: 0.8347 - val_loss: 0.3463 - val_accuracy: 0.9101
Epoch 5/20
47/47 [==============================] - 31s 648ms/step - loss: 0.4161 - accuracy: 0.8320 - val_loss: 0.3254 - val_accuracy: 0.8968
Epoch 6/20
47/47 [==============================] - 31s 653ms/step - loss: 0.3797 - accuracy: 0.8509 - val_loss: 0.2919 - val_accuracy: 0.9180
Epoch 7/20
47/47 [==============================] - 34s 718ms/step - loss: 0.3539 - accuracy: 0.8598 - val_loss: 0.2801 - val_accuracy: 0.9180